In [ ]:
!pip3 install selenium
!pip3 install webdriver-manager

In [ ]:
# Import libraries
import calendar
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

In [ ]:
# Define cazoo url
base_url = "https://www.cazoo.co.uk"

In [ ]:
# Create Pandas Dataframe
vehicle_df = pd.DataFrame(columns=['Make', 'Model', 'Price (£)', 'Milage', 'Registration Year', 'Registration Month', 'Gearbox', 'Fuel Type', 'N of Owners'])

In [ ]:
# Get response code
response = requests.get(base_url)
print(response.status_code)

In [ ]:
# Create page counter
page_no = 1

In [ ]:
# Loop for pages
for page in range(10):

    # Define url
    listings_url = f"https://www.cazoo.co.uk/cars/?page={page_no}"

    # Get page url and parse it
    response = requests.get(listings_url)
    soup = BeautifulSoup(response.text, features="html.parser")
    listings = soup.find_all(class_='vehicle-cardstyles__InfoWrap-sc-1bxv5iu-2 laxUtI')

    # Listing counter
    listing_count = 1

    # Print status
    print(f"Scraping from page {page_no}...")

    # Get a tags (urls) for each listing
    for listing in listings:

        # Print listing number
        print(f"listing {listing_count}...")

        # Find 'a' tags
        a_tags = listing.find_all('a')

        # In each url scrape the data
        for car_url in a_tags:

            # Parse html
            response = requests.get(base_url + car_url['href'])
            soup = BeautifulSoup(response.content, features='html.parser')

            # Get make and model
            try:
                make_model = soup.find('h1', class_='sc-yrk414-0 Abyeg').text.strip()
                make, model = make_model.split(' ', 1)
            except:
                make, model = "n/a"

            # Get price
            try:
                driver = webdriver.Chrome()
                driver.get(base_url + car_url['href'])
                time.sleep(1)
                price = driver.find_element(By.CSS_SELECTOR, "div.md\:px-l:nth-child(3) > p:nth-child(1) > small:nth-child(1) > small:nth-child(2) > strong:nth-child(1)").text
                price = int(price[1:].replace(',', ''))
            except:
                price = "n/a"

            # Get mileage
            try:
                mileage = int(soup.find_all('li', class_="sc-1qzxxja-0 sc-17oqk9n-0 cMysQa etVBLA")[0].text.strip().replace(' miles','').replace(',',''))
            except:
                mileage = "n/a"

            # Get registration year
            try:
                reg_year = soup.find_all('li', class_="sc-1qzxxja-0 sc-17oqk9n-0 cMysQa etVBLA")[1].text.strip()
                reg_year = int(reg_year[-4:])
            except:
                reg_year = "n/a"

            # Get registration month
            try:
                reg_month = soup.find_all('li', class_="sc-1qzxxja-0 sc-17oqk9n-0 cMysQa etVBLA")[1].text.strip()
                reg_month = reg_month[:3]
                reg_month = int(list(calendar.month_abbr).index(reg_month))
            except:
                reg_month = "n/a"

            # Get gearbox
            try:
                gearbox = soup.find_all('li', class_="sc-1qzxxja-0 sc-17oqk9n-0 cMysQa etVBLA")[2].text.strip()
            except:
                gearbox = "n/a"

            # Get fuel type
            try:
                fuel_type = soup.find_all('li', class_="sc-1qzxxja-0 sc-17oqk9n-0 cMysQa etVBLA")[3].text.strip()
            except:
                fuel_type = "n/a"

            # Get number of owners
            try:
                number_of_owners = soup.select("dl.divide-y:nth-child(2) > div:nth-child(3) > dd:nth-child(2)")[0]
                number_of_owners = int(number_of_owners.text)
            except:
                number_of_owners = "n/a"

            # Create row
            row = pd.Series({'Make': make,
                             'Model': model,
                             'Price (£)': price,
                             'Milage': mileage,
                             'Registration Year': reg_year,
                             'Registration Month': reg_month,
                             'Gearbox': gearbox,
                             'Fuel Type': fuel_type,
                             'N of Owners': number_of_owners
                            })

            # Add row to pandas data frame
            vehicle_df.loc[len(vehicle_df)] = row

        listing_count += 1
        if listing_count > 47:
            break

    # Go to next page
    page_no += 1

In [ ]:
vehicle_df.to_csv("cazoo_used_cars_dataset.csv")